In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly

In [2]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests

from bs4 import BeautifulSoup

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



# Data adquisition

The coordinates of the neighborhoods are extracted from the [Seville city hall webpage](https://www.sevilla.org/servicios/servicio-de-estadistica/callejero/datos-alfanumericos), and transformed in the file seville_hoods.csv

In [4]:
neighborhoods = pd.read_csv("seville_hoods.csv")
neighborhoods.tail()

,Unnamed: 0,Name,Latitude,Longitude
103,103,Aeropuerto Viejo,37.415896,-5.946192
104,104,"Palacio Congresos, Urbadiez, Entrepuentes, Jar...",37.396032,-5.936959
105,105,"Colores, Entreparques",37.392615,-5.873559
106,106,Valdezorras,37.427943,-5.914069
107,107,Palmete,37.377730,-5.931119


The coordinates of the city can be obtained with geopy 

In [5]:
address = 'Seville'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

The map of the city and the location of the neighborhoods are plotted

In [6]:
map_seville = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Name']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seville)  
    
map_seville

The credentials of Foursquare are stored

In [34]:
CLIENT_ID = '******' # your Foursquare ID
CLIENT_SECRET = '******' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ******
CLIENT_SECRET:******


A function to get the venues of each place is created and used

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
seville_venues = getNearbyVenues(names=neighborhoods['Name'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Bellavista
El Plantinar
El Cerro
Retiro Obrero
La Oliva
Bda. Pino Montano
Torreblanca
Isla de Garza
Tablada
Las Almenas
Triana Oeste
Los Remedios
El Cano, Los Bermejales
El Porvenir
Pedro Salvador, Las Palmeritas
Nervión
San Bartolomé
Doctor Barraquer, G. Renfe, Policlínico
Los Pájaros
San Pablo C
El Gordillo
El Tardón, El Carmen
Sector Sur, La Palmera, Reina Mercedes
Bami
Tiro de Línea, Santa Genoveva
Felipe II, Los Diez Mandamientos
Rochelambert
Feria
Macarena 3 Huertas, Macarena 5
El Rocío
Los Príncipes, La Fontanilla
Las Huertas
Santa María de Ordas, San Nicolás
Arbol Gordo
Santa Aurelia, Cantábrico, Atlántico, La Romería
Santa Cruz
Giralda Sur
La Plata
Triana Casco Antiguo
Museo
La Florida
San Lorenzo
La Paz, Las Golondrinas
Hermandades, La Carrasca
Polígono Norte
La Barzola
Los Arcos
Amate
Juan XXIII
Triana Este
San Vicente
Encarnación, Regina
Barrio León
San Roque
San Julián
Carretera de Carmona, María Auxiliadora, Fontanal
San José Obrero
San Gil
Cisneo Alto, Santa María de Gra

The data are verified

In [10]:
print(seville_venues.shape)
seville_venues.head()

(926, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bellavista,37.335018,-5.97233,Burger King,37.335343,-5.969593,Fast Food Restaurant
1,Bellavista,37.335018,-5.97233,Harley Davidson Sevilla,37.334349,-5.969614,Motorcycle Shop
2,Bellavista,37.335018,-5.97233,Pa Comé,37.334255,-5.969604,Café
3,Bellavista,37.335018,-5.97233,Super Bazar,37.334168,-5.969459,Miscellaneous Shop
4,Bellavista,37.335018,-5.97233,Seven Fitness,37.333658,-5.969776,Gym


The venues are grouped by neighborhood

In [11]:
seville_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()
seville_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bellavista,37.335018,-5.97233,Burger King,37.335343,-5.969593,Fast Food Restaurant
1,Bellavista,37.335018,-5.97233,Harley Davidson Sevilla,37.334349,-5.969614,Motorcycle Shop
2,Bellavista,37.335018,-5.97233,Pa Comé,37.334255,-5.969604,Café
3,Bellavista,37.335018,-5.97233,Super Bazar,37.334168,-5.969459,Miscellaneous Shop
4,Bellavista,37.335018,-5.97233,Seven Fitness,37.333658,-5.969776,Gym


In [12]:
print('There are {} uniques categories.'.format(len(seville_venues['Venue Category'].unique())))

There are 134 uniques categories.


Some places don't have enough venues to be representatives. A list with the neighborhoods with more than 5 results is made

In [13]:
seville_filtered = seville_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()
seville_filtered = seville_filtered[seville_filtered['Venue']>4]
seville_filtered.head()

,Venue
Neighborhood,
Alfalfa,62
Arbol Gordo,8
Arenal,24
Bami,5
Barrio León,7


In [14]:
seville_filtered.shape

(56, 1)

A data frame is created replacing the features with dummy variables. Then it will be  by neighborhoods and filtered (number of venues > 4)

In [15]:
# one hot encoding
seville_onehot = pd.get_dummies(seville_venues[['Venue Category']], prefix="", prefix_sep="")
columns = list(seville_onehot.columns)
columns.remove('Neighborhood')

# add neighborhood column back to dataframe
seville_onehot['Neighborhood'] = seville_venues['Neighborhood'] 
new_columns = ['Neighborhood'] + columns
seville_onehot = seville_onehot[new_columns]

seville_onehot.head()

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Boarding House,Bookstore,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Camera Store,Candy Store,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Garden,Gastropub,Gay Bar,Golf Course,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Karaoke Bar,Kids Store,Lake,Latin American Restaurant,Lebanese Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Palace,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Pizza Place,Plaza,Pool,Pub,Record Shop,Rental Car Location,Restaurant,Road,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Theme Park,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Wine Bar
0,Bellavista,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bellavista,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bellavista,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bellavista,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bellavista,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
seville_grouped = seville_onehot.groupby('Neighborhood').mean().reset_index()
seville_grouped = seville_grouped[seville_grouped['Neighborhood'].isin(list(seville_filtered.index))]
seville_onehot.shape

(926, 134)

A function to get the most common venues of each neighborhood is defined

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = seville_grouped['Neighborhood']

for ind in np.arange(seville_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(seville_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Alfalfa,Plaza,Tapas Restaurant,Restaurant,Ice Cream Shop,Clothing Store
1,Arbol Gordo,Tapas Restaurant,Park,Pizza Place,Spanish Restaurant,Athletics & Sports
2,Arenal,Spanish Restaurant,Ice Cream Shop,Tapas Restaurant,Restaurant,Wine Bar
4,Bami,Spanish Restaurant,Tapas Restaurant,Plaza,Soccer Field,Cupcake Shop
5,Barrio León,Tapas Restaurant,Camera Store,Pizza Place,Plaza,Spanish Restaurant


# Model

k-means clustering algorithm 

In [20]:
# set number of clusters
kclusters = 7

seville_grouped_clustering = seville_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seville_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 5, 0, 5, 0, 1, 4, 4, 0, 0], dtype=int32)

In [25]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()
seville_merged = neighborhoods[neighborhoods['Name'].isin(list(seville_filtered.index))]
seville_merged = seville_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Name')
seville_merged.head()

,Unnamed: 0,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,0,Bellavista,37.335018,-5.972330,1,Fast Food Restaurant,Gym,Motorcycle Shop,Café,Miscellaneous Shop
3,3,Retiro Obrero,37.402457,-5.979158,5,Tapas Restaurant,Ice Cream Shop,Bakery,Spanish Restaurant,Food
11,11,Los Remedios,37.374236,-5.999090,0,Mexican Restaurant,Gastropub,Pizza Place,Restaurant,Fast Food Restaurant
12,12,"El Cano, Los Bermejales",37.345470,-5.978850,0,Tapas Restaurant,Restaurant,Bar,Pizza Place,Spanish Restaurant
13,13,El Porvenir,37.371203,-5.982425,5,Tapas Restaurant,Spanish Restaurant,Coffee Shop,Bar,Seafood Restaurant


In [108]:
seville_merged.dropna(axis=0, inplace=True)
seville_merged["Cluster Labels"].astype(int)
seville_merged.shape

(56, 10)

Map with the classified neighborhoods

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(seville_merged['Latitude'], seville_merged['Longitude'], seville_merged['Name'], seville_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The different clusters

In [27]:
seville_merged.loc[seville_merged['Cluster Labels'] == 0, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,Los Remedios,Mexican Restaurant,Gastropub,Pizza Place,Restaurant,Fast Food Restaurant
12,"El Cano, Los Bermejales",Tapas Restaurant,Restaurant,Bar,Pizza Place,Spanish Restaurant
17,"Doctor Barraquer, G. Renfe, Policlínico",Supermarket,Ice Cream Shop,Sandwich Place,Beer Garden,Pub
22,"Sector Sur, La Palmera, Reina Mercedes",Pizza Place,Sandwich Place,College Quad,History Museum,Café
27,Feria,Tapas Restaurant,Café,Cocktail Bar,Mediterranean Restaurant,Pizza Place
35,Santa Cruz,Spanish Restaurant,Plaza,Restaurant,Tapas Restaurant,Cafeteria
36,Giralda Sur,Spanish Restaurant,American Restaurant,Gym,Hotel,Pizza Place
39,Museo,Hotel,Spanish Restaurant,Coffee Shop,Nightclub,Restaurant
40,La Florida,Bar,Café,Tapas Restaurant,Gastropub,Spanish Restaurant
51,"Encarnación, Regina",Tapas Restaurant,Hotel,Plaza,Café,Coffee Shop


In [28]:
seville_merged.loc[seville_merged['Cluster Labels'] == 1, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bellavista,Fast Food Restaurant,Gym,Motorcycle Shop,Café,Miscellaneous Shop
15,Nervión,Spanish Restaurant,Café,Coffee Shop,Plaza,Pub
24,"Tiro de Línea, Santa Genoveva",Café,Gastropub,Bakery,Gym / Fitness Center,Farmers Market
34,"Santa Aurelia, Cantábrico, Atlántico, La Romería",Café,Park,Plaza,Spanish Restaurant,Mexican Restaurant
64,Pino Flores,Basketball Court,Breakfast Spot,Convenience Store,Café,Candy Store


In [29]:
seville_merged.loc[seville_merged['Cluster Labels'] == 2, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,"Pedro Salvador, Las Palmeritas",Pharmacy,Tapas Restaurant,Pool,Brewery,Breakfast Spot
75,Villegas,Tapas Restaurant,Brewery,Bar,Breakfast Spot,Diner


In [30]:
seville_merged.loc[seville_merged['Cluster Labels'] == 3, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
16,San Bartolomé,Tapas Restaurant,Hotel,Performing Arts Venue,Spa,Bakery
71,La Calzada,Tapas Restaurant,Hotel,Restaurant,Indian Restaurant,Supermarket
82,Santa Catalina,Tapas Restaurant,Hotel,Spanish Restaurant,Hostel,Dessert Shop
93,"Tabladilla, La Estrella",Hotel,Breakfast Spot,Pizza Place,Spanish Restaurant,Gym
94,Huerta de la Salud,Hotel,Restaurant,Italian Restaurant,Train Station,Food


In [31]:
seville_merged.loc[seville_merged['Cluster Labels'] == 4, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
32,"Santa María de Ordas, San Nicolás",Pizza Place,Beer Garden,Fish & Chips Shop,Mediterranean Restaurant,Breakfast Spot
55,"Carretera de Carmona, María Auxiliadora, Fontanal",Ice Cream Shop,Fast Food Restaurant,Gastropub,Supermarket,Grocery Store
58,"Cisneo Alto, Santa María de Gracia",American Restaurant,Bar,Mediterranean Restaurant,Tapas Restaurant,Breakfast Spot
67,Heliópolis,Soccer Stadium,Tapas Restaurant,Mediterranean Restaurant,Seafood Restaurant,Spanish Restaurant
88,Zodiaco,American Restaurant,Tapas Restaurant,Spanish Restaurant,Mediterranean Restaurant,Brewery


In [32]:
seville_merged.loc[seville_merged['Cluster Labels'] == 5, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Retiro Obrero,Tapas Restaurant,Ice Cream Shop,Bakery,Spanish Restaurant,Food
13,El Porvenir,Tapas Restaurant,Spanish Restaurant,Coffee Shop,Bar,Seafood Restaurant
23,Bami,Spanish Restaurant,Tapas Restaurant,Plaza,Soccer Field,Cupcake Shop
28,"Macarena 3 Huertas, Macarena 5",Tapas Restaurant,Plaza,Coffee Shop,Event Space,Food
33,Arbol Gordo,Tapas Restaurant,Park,Pizza Place,Spanish Restaurant,Athletics & Sports
38,Triana Casco Antiguo,Tapas Restaurant,Spanish Restaurant,Bar,Restaurant,Brewery
41,San Lorenzo,Tapas Restaurant,Gay Bar,Spanish Restaurant,Indian Restaurant,Bar
49,Triana Este,Tapas Restaurant,Grocery Store,Seafood Restaurant,Spanish Restaurant,Pub
87,"San Carlos, Tartessos",Tapas Restaurant,Spanish Restaurant,Cocktail Bar,Café,Fast Food Restaurant
91,San Diego,Tapas Restaurant,Brewery,Candy Store,Bar,Beer Bar


In [33]:
seville_merged.loc[seville_merged['Cluster Labels'] == 6, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
29,El Rocío,Fried Chicken Joint,Dessert Shop,Latin American Restaurant,Restaurant,Diner
43,"Hermandades, La Carrasca",Fried Chicken Joint,Bakery,Latin American Restaurant,Restaurant,Diner
63,Santa Clara,Grocery Store,Flower Shop,Pool,Fish Market,Theme Park
83,Las Avenidas,Tapas Restaurant,Wine Bar,Latin American Restaurant,Grocery Store,Dessert Shop
85,"El Torrejón, El Cerezo",Wine Bar,Tapas Restaurant,Pizza Place,Dessert Shop,Restaurant
